In [2]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from ipywidgets import interact
import json

In [3]:
# Read CSV file
df = pd.read_csv('Data/us-traffic-congestions-2016-2022/us_congestion_2016_2022/us_congestion_2016_2022.csv')

# Drop records without timestamps
df = df.dropna(subset=['StartTime','EndTime'])

df.shape

C:\Users\Raul\AppData\Local\Temp\ipykernel_35104\3848489488.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data/us-traffic-congestions-2016-2022/us_congestion_2016_2022/us_congestion_2016_2022.csv')


(33273373, 30)

In [4]:
# Adding additional fields and transformations
# Break down Weather_Conditions with more than one value
df[['Weather_Conditions','Weather_Conditions_Other']] = df['Weather_Conditions'].str.split('/',n=1, expand=True)

# Break down timestamp
df['StartTime'] = pd.to_datetime(df['StartTime'], utc=True)
df['EndTime'] = pd.to_datetime(df['EndTime'], utc=True)

df['Start_Hour'] = df['StartTime'].dt.hour
df['Start_Minute'] = df['StartTime'].dt.minute
df['Start_Day'] = df['StartTime'].dt.day
df['Start_Month'] = df['StartTime'].dt.month
df['Start_Year'] = df['StartTime'].dt.year
df['Start_DayOfWeek'] = df['StartTime'].dt.weekday

# Calculate duration in minutes
df['Duration'] = df['EndTime'] - df['StartTime']
df['Duration_min'] = df['Duration'].dt.total_seconds() / 60
df['Duration_min'] = df['Duration_min'].astype('float')

df.shape

(33273373, 39)

In [7]:
# drop unnecessary columns and fill missing values
df = df.drop(columns=['ID'
                      ,'StartTime'
                      ,'EndTime'
                      ,'Duration'
                      ,'Description'
                      ,'Congestion_Speed'
                      ,'Weather_Event'
                      ,'Street'
                      ,'ZipCode'
                      ,'LocalTimeZone'
                      ,'WeatherStation_AirportCode'
                      ,'WeatherTimeStamp'
                      ,'WindChill(F)'
                      ,'Pressure(in)'
                      ,'WindDir'])
df.fillna({'WindSpeed(mph)': 0,
           'Precipitation(in)': 0,
           'Visibility(mi)': df['Visibility(mi)'].median(),
           'Temperature(F)': df['Temperature(F)'].median(),
           'Humidity(%)': df['Humidity(%)'].median(),
           }, inplace=True)
df.head()

,Severity,Start_Lat,Start_Lng,Distance(mi),DelayFromTypicalTraffic(mins),DelayFromFreeFlowSpeed(mins),City,County,State,Country,...,Precipitation(in),Weather_Conditions,Weather_Conditions_Other,Start_Hour,Start_Minute,Start_Day,Start_Month,Start_Year,Start_DayOfWeek,Duration_min
0,0,42.410881,-71.147995,3.99,0.0,4.0,Arlington,Middlesex,MA,US,...,0.0,Fair,None,22,54,10,11,2020,1,95.450000
1,0,42.512131,-71.027992,0.75,1.0,1.0,Lynnfield,Essex,MA,US,...,0.0,Fair,None,22,53,10,11,2020,1,40.716667
2,1,42.272388,-71.068001,4.28,0.0,7.0,Dorchester Center,Suffolk,MA,US,...,0.0,Fair,None,22,49,10,11,2020,1,50.650000
3,0,42.356911,-71.066254,0.38,0.0,2.0,Boston,Suffolk,MA,US,...,0.0,Mostly Cloudy,None,22,58,10,11,2020,1,41.650000
4,1,42.377628,-71.120079,3.55,0.0,5.0,Cambridge,Middlesex,MA,US,...,0.0,Mostly Cloudy,None,22,46,10,11,2020,1,53.650000


In [8]:
df.to_csv('Data/us-traffic-congestions-2016-2022-cleaned.csv', index=False, encoding='utf-8')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33273373 entries, 0 to 33304198
Data columns (total 24 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   Severity                       int64  
 1   Start_Lat                      float64
 2   Start_Lng                      float64
 3   Distance(mi)                   float64
 4   DelayFromTypicalTraffic(mins)  float64
 5   DelayFromFreeFlowSpeed(mins)   float64
 6   City                           object 
 7   County                         object 
 8   State                          object 
 9   Country                        object 
 10  Temperature(F)                 float64
 11  Humidity(%)                    float64
 12  Visibility(mi)                 float64
 13  WindSpeed(mph)                 float64
 14  Precipitation(in)              float64
 15  Weather_Conditions             object 
 16  Weather_Conditions_Other       object 
 17  Start_Hour                     int32  
 18  Start

In [10]:
state_codes = pd.read_csv('Data/state_codes.csv')
state_codes['code'] = state_codes['code'].astype(str).str.zfill(2)
state_codes.head()

,state,code
0,AL,01
1,AK,02
2,AZ,04
3,AR,05
4,CA,06


In [11]:
county_codes = pd.read_csv('Data/county_codes.csv')
county_codes.head()

,geocode,StateCode,CountyCode,County,State
0,0500000US01001,1,1,Autauga,AL
1,0500000US01003,1,3,Baldwin,AL
2,0500000US01005,1,5,Barbour,AL
3,0500000US01007,1,7,Bibb,AL
4,0500000US01009,1,9,Blount,AL


In [ ]:
state_year_data = df.groupby(['State','Start_Year']).size().reset_index(name='Count')
state_year_data = state_year_data.merge(state_codes, left_on='State', right_on='state', how='inner')
state_year_data = state_year_data.drop(columns=['state'])
state_year_data


,State,Start_Year,Count,code
0,AL,2016,450,01
1,AL,2017,35984,01
2,AL,2018,100399,01
3,AL,2019,69931,01
4,AL,2020,29563,01
...,...,...,...,...
320,WY,2018,29434,56
321,WY,2019,46552,56
322,WY,2020,18772,56
323,WY,2021,19880,56


In [29]:
state_year_data.to_csv('Data/state_year_data.csv', index=False, encoding='utf-8')

In [13]:
county_year_data = df.groupby(['State','County','Start_Year']).size().reset_index(name='Count')
county_year_data = county_year_data.merge(county_codes, left_on='County', right_on='County', how='inner')
county_year_data.drop(columns=['StateCode','CountyCode','State_y'])
county_year_data

,State_x,County,Start_Year,Count,geocode,StateCode,CountyCode,State_y
0,AL,Autauga,2017,500,0500000US01001,1,1,AL
1,AL,Autauga,2018,1192,0500000US01001,1,1,AL
2,AL,Autauga,2019,1151,0500000US01001,1,1,AL
3,AL,Autauga,2020,242,0500000US01001,1,1,AL
4,AL,Autauga,2021,473,0500000US01001,1,1,AL
...,...,...,...,...,...,...,...,...
94027,WY,Weston,2018,174,0500000US56045,56,45,WY
94028,WY,Weston,2019,145,0500000US56045,56,45,WY
94029,WY,Weston,2020,89,0500000US56045,56,45,WY
94030,WY,Weston,2021,446,0500000US56045,56,45,WY


In [26]:
# with open('Data/us-states.json') as f:
#     us_states = json.load(f)

with open('Data/us-states.json') as f:
    us_states = json.load(f)

In [24]:
location_counts = df.groupby(['Start_Year','Start_Lat','Start_Lng']).size().reset_index(name='Count')
location_counts.sort_values(by='Count', ascending=False, inplace=True)
location_counts
filtered_location_counts = location_counts[location_counts['Count'] > 10]
filtered_location_counts



,Start_Year,Start_Lat,Start_Lng,Count
3791714,2019,29.455536,-89.663963,3304
3662593,2018,48.999859,-122.754623,3141
4944526,2019,40.695107,-73.998497,2892
5624108,2020,29.350363,-89.529289,2202
7603792,2021,40.731831,-73.996651,2054
...,...,...,...,...
7654263,2021,41.129822,-73.463150,11
3418416,2018,42.860012,-70.873177,11
7179443,2021,34.047642,-118.324570,11
1397117,2017,41.661324,-72.668747,11


In [40]:
filtered_location_counts.to_csv('Data/filtered_location_counts.csv', index=False, encoding='utf-8')

In [ ]:
# Create Map for specific year
def create_map(year):
    # Filtered data for selected year
    filtered_data = state_year_data[state_year_data['Start_Year'] == year]
    # filtered_data = county_year_data[county_year_data['Start_Year'] == year]

    # Prepare data for the heatmap
    filtered_heat_data = filtered_location_counts[filtered_location_counts['Start_Year'] == year]   
    heat_data = filtered_heat_data[['Start_Lat', 'Start_Lng','Count']].values.tolist()

    # initialize the map
    m = folium.Map(location=[37.8, -96], zoom_start=4.1, tiles="Cartodb Positron")

    # Add Choropleth layer
    folium.Choropleth(
        name='Choropleth',
        geo_data=us_states,
        data=filtered_data,
        columns=['code', 'Count'],
        key_on='feature.properties.STATE',
        fill_color='YlGn',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f'Traffic Incidents in {year}',
        highlight=True,
        use_jenks=True
    ).add_to(m)

    HeatMap(
        name='Heat Map',
        data=heat_data, 
        radius=8, 
        blur=4,
        min_opacity=0.6, 
        show=False,
    ).add_to(m)

    # Add Layer Control
    folium.LayerControl().add_to(m)

    return m

# Create Interactive map
interact(create_map, year=sorted(state_year_data['Start_Year'].unique()))

interactive(children=(Dropdown(description='year', options=(np.int32(2016), np.int32(2017), np.int32(2018), np…

<function __main__.create_map(year)>